In [1]:
# Setup the weather app
#1. import dependencies 
import datetime as dt
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [2]:
# Setup the Database

engine = create_engine("sqlite:///hawaii.sqlite")

# Use automap_base() to put the database into classes and .prepare() to reflect the tables into SQLAlchemy
Base = automap_base()
Base.prepare(engine, reflect=True)

# Save our references to each table - Create a variable for each of the classes so we can reference them later

Measurement = Base.classes.measurement
Station = Base.classes .station

# Create a session link from Python to the Database

session = Session(engine)

# Create a flask application, being sure to pass __name__ in the app variable - we are putting the flask object into the app variable
app = Flask(__name__)



In [3]:
# Define our route - what to do when a user hits the index route - in this case this is the homepage - this is a static route
@app.route('/')
def welcome():
    return (
    '''
    Welcome to the Climate Analysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    ''')



# Create the precipitation route

## Calculate the date for one year from 'now'
First we add the code to calculate the date one year ago

In [4]:
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
prev_year    

datetime.date(2016, 8, 23)

## Query for the Date and Precipitation from the Measurement Table
We can find out the referenced names of the columns we need in the table using the below
### Find column headings of the table `Measurement`

In [5]:
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2252bccea88>,
 'date': '2010-01-01',
 'tobs': 65.0,
 'id': 1,
 'station': 'USC00519397',
 'prcp': 0.08}

### Write the query to get `date` and `prcp` and filter for all data for the 

In [6]:
precipitation = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()
precipitation

[('2016-08-23', 0.0),
 ('2016-08-24', 0.08),
 ('2016-08-25', 0.08),
 ('2016-08-26', 0.0),
 ('2016-08-27', 0.0),
 ('2016-08-28', 0.01),
 ('2016-08-29', 0.0),
 ('2016-08-30', 0.0),
 ('2016-08-31', 0.13),
 ('2016-09-01', 0.0),
 ('2016-09-02', 0.0),
 ('2016-09-03', 0.0),
 ('2016-09-04', 0.03),
 ('2016-09-05', None),
 ('2016-09-06', None),
 ('2016-09-07', 0.05),
 ('2016-09-08', 0.0),
 ('2016-09-09', 0.03),
 ('2016-09-10', 0.0),
 ('2016-09-11', 0.05),
 ('2016-09-12', 0.0),
 ('2016-09-13', 0.02),
 ('2016-09-14', 1.32),
 ('2016-09-15', 0.42),
 ('2016-09-16', 0.06),
 ('2016-09-17', 0.05),
 ('2016-09-18', 0.0),
 ('2016-09-19', 0.0),
 ('2016-09-20', 0.0),
 ('2016-09-21', 0.0),
 ('2016-09-22', 0.02),
 ('2016-09-23', 0.0),
 ('2016-09-24', 0.0),
 ('2016-09-25', 0.0),
 ('2016-09-26', 0.06),
 ('2016-09-27', 0.02),
 ('2016-09-28', 0.0),
 ('2016-09-29', 0.0),
 ('2016-09-30', 0.0),
 ('2016-10-01', 0.0),
 ('2016-10-02', 0.0),
 ('2016-10-03', 0.0),
 ('2016-10-04', 0.0),
 ('2016-10-05', 0.0),
 ('2016-10-06'

The precipitation data is in a format of (date,prcp), but instead we want to format our results in a JSON structured file since JSON is a more widely used format 

### Use the `jsonify()` function to convert to JSON

We can use the jsonify() function which converts a dictionary into a JSON file. jsonify() is a helper method provided by Flask to properly return JOSN data. So we use jsonify() in the route function as a return object

But our current data is in the (date,prcp) format. To change this into a dictionary we can use a for loop to go through the list and change it to a dict format (date:prcp)

### use dict comprehension
Instead of a for loop, we can use a dict comprehension - similar to that of a list comprehension, but instead of using square brackets [ ] we use curly brackets { } to show the result as a dict

In [7]:
precip = {date: prcp for date, prcp in precipitation}
print(precip)


{'2016-08-23': 0.7, '2016-08-24': 1.45, '2016-08-25': 0.11, '2016-08-26': 0.01, '2016-08-27': None, '2016-08-28': 2.07, '2016-08-29': 0.9, '2016-08-30': 0.05, '2016-08-31': 2.46, '2016-09-01': 0.01, '2016-09-02': 0.03, '2016-09-03': 1.0, '2016-09-04': 0.44, '2016-09-05': 0.18, '2016-09-06': 1.0, '2016-09-07': 1.35, '2016-09-08': 0.15, '2016-09-09': 0.35, '2016-09-10': 1.16, '2016-09-11': 0.6, '2016-09-12': 1.04, '2016-09-13': 1.2, '2016-09-14': 6.7, '2016-09-15': 3.35, '2016-09-16': 0.61, '2016-09-17': 0.23, '2016-09-18': 0.42, '2016-09-19': 0.25, '2016-09-20': 0.43, '2016-09-21': 1.02, '2016-09-22': 0.75, '2016-09-23': 0.33, '2016-09-24': 0.27, '2016-09-25': 0.04, '2016-09-26': 1.02, '2016-09-27': 1.0, '2016-09-28': 0.05, '2016-09-29': 1.49, '2016-09-30': 0.38, '2016-10-01': 1.02, '2016-10-02': 0.61, '2016-10-03': 0.46, '2016-10-04': 3.46, '2016-10-05': 0.81, '2016-10-06': 0.04, '2016-10-07': 0.01, '2016-10-08': 0.04, '2016-10-09': 0.0, '2016-10-10': 0.0, '2016-10-11': 0.35, '2016-10-

In [8]:
# Completed precipitation route

@app.route('/api/v1.0/precipitation')
def precipitation():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    precipitation = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= prev_year).all()
    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip)

# Create the Stations Route
We can start by defining the route:

    @app.route('/api/v1.0/stations')
    def stations():
        return

We now want the stations data by querying the stations table and the stations column

In [9]:
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x2252bce6108>,
 'id': 1,
 'name': 'WAIKIKI 717.2, HI US',
 'longitude': -157.8168,
 'latitude': 21.2716,
 'station': 'USC00519397',
 'elevation': 3.0}

We now know the column name is `'station'`

In [10]:
results = session.query(Station.station).all()
results

[('USC00519397'),
 ('USC00513117'),
 ('USC00514830'),
 ('USC00517948'),
 ('USC00518838'),
 ('USC00519523'),
 ('USC00519281'),
 ('USC00511918'),
 ('USC00516128')]

### Use `function np.ravel()` to create a list
Again we want to `jsonify()` this data, so we have to convert this data into a one-dimensional array and then into a list

We can do this by using `funciton np.ravel()` to unravel the results data and then put them into a list using the `list()` function

In [11]:
stations = np.ravel(results)
stations

array(['USC00519397', 'USC00513117', 'USC00514830', 'USC00517948',
       'USC00518838', 'USC00519523', 'USC00519281', 'USC00511918',
       'USC00516128'], dtype='<U11')

In [12]:
stations = list(np.ravel(results))
stations

['USC00519397',
 'USC00513117',
 'USC00514830',
 'USC00517948',
 'USC00518838',
 'USC00519523',
 'USC00519281',
 'USC00511918',
 'USC00516128']

We can now jsonify this data in the completed stations route

In [13]:
# Completed stations route

@app.route('/api/v1.0/stations')
def stations():
    results = session.query(Station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations=stations)

You may notice here that to return our list as JSON, we need to add stations=stations. This formats our list into JSON. If you'd like to read more about it, checkout the Flask documentation - https://flask.palletsprojects.com/en/1.1.x/api/#flask.json.jsonify

# Create the Monthly Temperature Route
This route will show us all of the temperature observations for the previous year, just for the primary station which we determined by doing a count of values that are in the Measurement.station table.column

In [14]:
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

This shows that `'USC00519281'` has the most so we will filter to just this station

In [15]:
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
results = session.query(Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    filter(Measurement.date >= prev_year).all()
results

[(77.0),
 (77.0),
 (80.0),
 (80.0),
 (75.0),
 (73.0),
 (78.0),
 (77.0),
 (78.0),
 (80.0),
 (80.0),
 (78.0),
 (78.0),
 (78.0),
 (73.0),
 (74.0),
 (80.0),
 (79.0),
 (77.0),
 (80.0),
 (76.0),
 (79.0),
 (75.0),
 (79.0),
 (78.0),
 (79.0),
 (78.0),
 (78.0),
 (76.0),
 (74.0),
 (77.0),
 (78.0),
 (79.0),
 (79.0),
 (77.0),
 (80.0),
 (78.0),
 (78.0),
 (78.0),
 (77.0),
 (79.0),
 (79.0),
 (79.0),
 (79.0),
 (75.0),
 (76.0),
 (73.0),
 (72.0),
 (71.0),
 (77.0),
 (79.0),
 (78.0),
 (79.0),
 (77.0),
 (79.0),
 (77.0),
 (78.0),
 (78.0),
 (78.0),
 (78.0),
 (77.0),
 (74.0),
 (75.0),
 (76.0),
 (73.0),
 (76.0),
 (74.0),
 (77.0),
 (76.0),
 (76.0),
 (74.0),
 (75.0),
 (75.0),
 (75.0),
 (75.0),
 (71.0),
 (63.0),
 (70.0),
 (68.0),
 (67.0),
 (77.0),
 (74.0),
 (77.0),
 (76.0),
 (76.0),
 (75.0),
 (76.0),
 (75.0),
 (73.0),
 (75.0),
 (73.0),
 (75.0),
 (74.0),
 (75.0),
 (74.0),
 (75.0),
 (73.0),
 (75.0),
 (73.0),
 (73.0),
 (74.0),
 (70.0),
 (72.0),
 (70.0),
 (67.0),
 (67.0),
 (69.0),
 (70.0),
 (68.0),
 (69.0),
 (69.0),
 

Similar to the stations route, we need to unravel the results into a on-dimensional array and convert the array into a list. Then we jsonify the list and return it to get our resulting JSON

In [ ]:
# Completed Monthly Temperature Route
@app.route("/api/v1.0/tobs")
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    results = session.query(Measurement.tobs).\
      filter(Measurement.station == 'USC00519281').\
      filter(Measurement.date >= prev_year).all()
    temps = list(np.ravel(results))
    return jsonify(temps=temps)

# Statistics Route
## Adding in a start and end date into the route
For this route, we will have the route require a start and end date to retrive data

we add this by having our route setup in the following format:

    @app.route("/api/v1.0/temp/<start>")
    @app.route("/api/v1.0/temp/<start>/<end>")

The function will also reflect this and also have added parameters: a `start` parameter and an `end` parameter

To start we will have them set to `None`

    def stats(start=None, end=None):
        return

### Create the Query to find the Min ave and max temps
Creating a query to select the minimum, average and max temperatures from the SQLite database and putting it into a list called `sel`

In [18]:
sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
print(sel)

[<sqlalchemy.sql.functions.min at 0x2252bdd8a48; min>, <sqlalchemy.sql.functions.Function at 0x2252bdd8bc8; avg>, <sqlalchemy.sql.functions.max at 0x2252bdd8d48; max>]


We can apply this query to the function we defined above, but because we need to determine the starting and end date we need to use an `if-not` statement to the function

So if we don't have an end date and we just have the one date, 

In [23]:
# Completed Stats for Date Range Route
@app.route("/api/v1.0/temp/<start>")
@app.route("/api/v1.0/temp/<start>/<end>")
def stats(start=None, end=None):

    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).\
            filter(Measurement.date <= end).all()
        temps = list(np.ravel(results))
        return jsonify(temps)

    results = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
    temps = list(np.ravel(results))
    return jsonify(temps=temps)

We can test our temp stats route with the following

    /api/v1.0/temp/2017-06-01/2017-06-30